In [1]:
from pyspark.sql import SparkSession    

In [2]:
spark = SparkSession.builder.appName('Logistic').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
data = spark.read.format('libsvm').load('Spark_for_Machine_Learning/1.Logistic_Regression/sample_libsvm_data.txt')

In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_log_reg_model = LogisticRegression()

In [7]:
fitted_logreg_model = my_log_reg_model.fit(data)

In [12]:
log_summary = fitted_logreg_model.summary

In [14]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [15]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126974...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170280...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200599...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695573...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102630...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

# Using training set for the model

In [16]:
train, test = data.randomSplit([0.7,0.3])

In [19]:
final_model = LogisticRegression()

In [20]:
fit_final = final_model.fit(train)

# Evaluating the predictions    

In [21]:
prediction_and_labels = fit_final.evaluate(test)

In [22]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[30.0828283366966...|[0.99999999999991...|       0.0|
|  0.0|(692,[124,125,126...|[31.1567023293646...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[25.7386747838468...|[0.99999999999336...|       0.0|
|  0.0|(692,[126,127,128...|[26.3598000799543...|[0.99999999999643...|       0.0|
|  0.0|(692,[126,127,128...|[30.0424158553004...|[0.99999999999991...|       0.0|
|  0.0|(692,[126,127,128...|[21.1452723698515...|[0.99999999934427...|       0.0|
|  0.0|(692,[127,128,129...|[18.6132157139607...|[0.99999999175134...|       0.0|
|  0.0|(692,[153,154,155...|[13.3860943480635...|[0.99999846364201...|       0.0|
|  0.0|(692,[154,155,156...|[8.42970196554226...|[0.99978176110100...|       0.0|
|  0.0|(692,[155

In [23]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [24]:
my_eval = BinaryClassificationEvaluator()

In [27]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [28]:
my_final_roc

1.0

# Trying SQL

In [30]:
type(data)

pyspark.sql.dataframe.DataFrame

In [31]:
#Register the DataFrame as a SQL temporary view
data.createOrReplaceTempView("data")

In [32]:
sqlDF = spark.sql("SELECT * FROM data")
sqlDF.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [33]:
sqlDF = spark.sql("SELECT count(features) FROM data as count")
sqlDF.show()

+---------------+
|count(features)|
+---------------+
|            100|
+---------------+

